# Ejercicio 1 - Manejo de Datos con NLP

In [1]:
# Importo la librería de trabajo de Hugging Face (transformers) y lo que necesito
from transformers import pipeline, AutoTokenizer
# Importo pandas para trabajar con DataFrames
import pandas as pd
# Otras importanciones para limpieza de texto
from rapidfuzz import process
import unicodedata, re

c:\Users\chris\OneDrive\Documentos\SENASoft\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carga de trabajo

In [2]:
# Creo el DataFrame principal
df = pd.read_csv('../data/personas.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   nombre    21 non-null     object
 1   apellido  21 non-null     object
 2   edad      21 non-null     int64 
 3   sueldo    21 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 804.0+ bytes


In [ ]:
# Asigno un modelo preentrenado para trabajar, especializado en clasificar texto
ai = pipeline(
    "zero-shot-classification",
    model="joeddav/xlm-roberta-large-xnli"
)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


## Funciones que usará el modelo

In [ ]:
# Función para quitar tildes, símbolos o demás
def limpiar_texto(texto: str) -> str:
    """limpiar_texto
    
    Se encarga de normalizar el texto,
    concatena cada caracter sin tildes,
    permite solo letras, números y espacios
    y quita los espacios de más.

    Args:
        texto (str): texto de cualquier longitud

    Returns:
        str: texto limpio para trabajar
    """

    # Lo paso a minusculas
    texto = texto.lower()
    # Lo normalizo
    texto = unicodedata.normalize("NFKD", texto)
    # Concateno los caracteres sin tildes
    texto = "".join([c for c in texto if not unicodedata.combining(c)])
    # Permito solamente números, letras y espacios
    texto = re.sub(r"[^a-z0-9\s]", " ", texto)
    # Quito los espacios sobrantes y retorno
    return re.sub(r"\s+", " ", texto).strip()



limpiar_texto("María está  mir_ndo   las esTRellas con  un niño")

'maria esta mir ndo las estrellas con un nino'

In [ ]:
# Función que recorre el texto en busca de una coincidencia
def extraer_nombre(texto: str, df: pd.DataFrame):
    """extraer_nombre
    
    Convierte los nombres de un DataFrame en
    una lista para recorrer, cada nombre se
    limpia (normaliza) y se recorre la lista
    en busca de una coincidencia

    Args:
        texto (str): texto en lenguaje humano
        df (pd.DataFrame): conjunto de datos

    Returns:
        str: texto con la coincidencia encontrada
    """

    # Lista de nombres
    nombres = df["nombre"].tolist()
    # Se limpia cada nombre
    nombres = [limpiar_texto(n) for n in nombres]
    # Se limpia el texto del parámetro
    texto = limpiar_texto(texto)
    # Se recorre cada palabra para buscar una coincidencia
    for i, n in enumerate(nombres):
        if n in texto:
            return df["nombre"].iloc[i]
        
    return "Comprueba el nombre que escribiste, o es posible que no exista en los registros"



extraer_nombre("está con laura en el parque", df)

'Laura'

In [ ]:
# Función para buscar sueldo por nombre
def buscar_sueldo_nombre(nombre: str, df: pd.DataFrame):
    """buscar_sueldo_nombre
    
    Compara un nombre con los datos de un 
    DataFrame para buscar el salario

    Args:
        nombre (str): nombre de una persona
        df (pd.DataFrame): conjunto de datos

    Returns:
        int: sueldo de la persona
    """
    persona = df.loc[df["nombre"].str.lower() == nombre.lower(), ["nombre", "sueldo"]]
    return persona['sueldo'].values[0] if not persona.empty else f"No se encontró ninguna persona con el nombre: {nombre.capitalize()}"



buscar_sueldo_nombre("maría", df)

np.int64(3200)

In [ ]:
# Función que busca el máximo rango de edad
def persona_mayor(df: pd.DataFrame):
    """persona_mayor

    Saca la edad máxima encontrada en 
    el DataFrame

    Args:
        df (pd.DataFrame): conjunto de datos

    Returns:
        DataFrame: conjunto de datos de la persona
    """
    edad_max = df["edad"].max()
    return df.loc[df["edad"] == edad_max]



persona_mayor(df)

,nombre,apellido,edad,sueldo
6,Andrés,García,45,4500


In [ ]:
# Función que saca el promedio de los sueldos registrados
def promedio(df: pd.DataFrame):
    return df["sueldo"].median()



promedio(df)

np.float64(3300.0)

## Función principal con el LLM funcionando

In [26]:
# Creo la función principal que clasificará y llamara a una función u otra dependiendo donde se clasifique
def responder(pregunta: str, df: pd.DataFrame):
    """responder
    
    Clasifica el lenguaje narutal en unas etiquetas
    para determinar qué función llamará:
        - buscar_sueldo_nombre()
        - persona_mayor()
        - promedio
    Si no entra en ninguna categoría se toma como 'otro'
    y le informa que no puede hacer esa consulta.

    Args:
        pregunta (str): texto en lenguaje humano
        df (pd.DataFrame): conjunto de datos

    Returns:
        str: cadena de texto que concatena la respuesta de la función
    """

    # Etiquetas para el clasificador
    etiquetas = ["sueldo persona", "persona con mas edad", "mas edad", "persona mas vieja", "promedio sueldo", "otro"]

    # Modelo clasificador
    clasificacion = ai(pregunta, etiquetas)

    # Condicionales de entrada para cada función
    if clasificacion['labels'][0] == "sueldo persona": #type:ignore
        nombre = extraer_nombre(pregunta, df)
        return f"El sueldo de {nombre} es: {buscar_sueldo_nombre(nombre, df)}"
    
    elif clasificacion['labels'][0] in etiquetas[1:4]:  #type:ignore
        return f"Te daré una lista con los datos que encontré: {persona_mayor(df)}"
    
    elif clasificacion['labels'][0] == "promedio sueldo": #type:ignore
        return f"El promedio de sueldos en mis registros es de: {promedio(df)}"
    
    # Condición si no corresponde con una clasificación adecuada
    else:
        return "Parece que tu solicitud no puedo comprobarla en este momento o no está en mis capacidades"



# Prueba de funcionamiento
responder("dime el nombre de la persona con más edad", df)

'Te daré una lista con los datos que encontré:    nombre apellido  edad  sueldo\n6  Andrés   García    45    4500'